In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.impute import SimpleImputer 
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, classification_report
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

### Modelo Red Neuronal

In [3]:
dataset = pd.read_csv("../data/raw/dataset_preparado.csv")
dataset.head()

,mora,atraso,vivienda,edad,dias_lab,exp_sf,nivel_ahorro,ingreso,linea_sf,deuda_sf,score,zona,clasif_sbs,nivel_educ
0,0,235,1,30.0,3748.0,93.000000,5.0,3500.0,11987.241038,0.00,214.0,14,4.0,3
1,0,18,1,32.0,4598.0,9.000000,12.0,900.0,1824.670000,1933.75,175.0,12,1.0,2
2,0,0,1,26.0,5148.0,8.000000,2.0,2400.0,2797.380000,188.29,187.0,14,0.0,3
3,0,0,1,36.0,5179.0,20.000000,12.0,2700.0,11987.241038,0.00,187.0,1,0.0,2
4,0,0,1,46.0,3960.0,32.401583,1.0,3100.0,2000.000000,11010.65,189.0,14,0.0,2


In [4]:
X = dataset.drop('mora', axis=1)  
y = dataset['mora']  
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

### Calcular el StandarScaler

In [6]:
std_scaler = StandardScaler()
std_scaler.fit(X_train) #calcular los valores para el scaler

StandardScaler()

In [7]:
import pickle  
with open('../artifacts/std_scaler.pkl', 'wb') as f:
    pickle.dump(std_scaler, f)

In [8]:
X_train_std=std_scaler.transform(X_train)

### Modelo No.1 Red Neuronal

In [9]:
# Crear el modelo secuencial
model = Sequential()
model.add(Dense(units=64, activation='relu', input_dim=X_train.shape[1]))  # 64 neuronas, función de activación ReLU
# Capa oculta adicional
model.add(Dense(units=32, activation='relu'))
# Capa de salida (para clasificación binaria: 1 o 0)
model.add(Dense(units=1, activation='sigmoid'))  # Función sigmoide para salida binaria (mora o no mora)
# Compilar el modelo
model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])


c:\Users\karen\anaconda4\envs\venv-churning-model\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [10]:
# Compilar el modelo
model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])
# Entrenar el modelo con 'X_train_std'
history = model.fit(X_train_std, y_train, epochs=30, batch_size=32, validation_split=0.2, verbose=1)

Epoch 1/30
168/168 ━━━━━━━━━━━━━━━━━━━━ 7s 9ms/step - accuracy: 0.6569 - loss: 0.6148 - val_accuracy: 0.7403 - val_loss: 0.5423
Epoch 2/30
168/168 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - accuracy: 0.7365 - loss: 0.5448 - val_accuracy: 0.7448 - val_loss: 0.5393
Epoch 3/30
168/168 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.7483 - loss: 0.5329 - val_accuracy: 0.7485 - val_loss: 0.5350
Epoch 4/30
168/168 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.7443 - loss: 0.5233 - val_accuracy: 0.7560 - val_loss: 0.5302
Epoch 5/30
168/168 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.7380 - loss: 0.5343 - val_accuracy: 0.7470 - val_loss: 0.5298
Epoch 6/30
168/168 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms/step - accuracy: 0.7467 - loss: 0.5211 - val_accuracy: 0.7560 - val_loss: 0.5247
Epoch 7/30
168/168 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.7433 - loss: 0.5279 - val_accuracy: 0.7545 - val_loss: 0.5243
Epoch 8/30
168/168 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - accuracy: 0.7483 - loss: 0.5145 - val_accuracy: 0

In [ ]:
# Evaluar el modelo en el conjunto de prueba escalado
loss, accuracy = model.evaluate(X_test_std, y_test)
print(f"Loss: {loss}")
print(f"Accuracy: {accuracy}")

In [ ]:
X_test_std = std_scaler.transform(X_test)
# Generar predicciones en el conjunto de prueba
y_pred = model.predict(X_test_std)
# Convertir las probabilidades en predicciones binarias con un umbral de 0.5
y_pred_bin = (y_pred > 0.5).astype(int)  # Si la probabilidad es mayor a 0.5, asigna 1, sino 0

In [ ]:
accuracy = accuracy_score(y_test, y_pred_bin)
print(f'Accuracy: {accuracy}')

### Modelo No.2

In [ ]:
rf_model_1 = RandomForestClassifier(n_estimators=100, max_depth=10, min_samples_split=2, random_state=42)
rf_model_1.fit(X_train_std, y_train)
y_pred_1 = rf_model_1.predict(X_test_std)
accuracy_1 = accuracy_score(y_test, y_pred_1)
print(f'Configuración 1 - Accuracy: {accuracy_1:.4f}')
print("Classification Report Configuración 1:")
print(classification_report(y_test, y_pred_1))

### Modelo No.3

In [ ]:
log_reg_model = LogisticRegression(random_state=42, 
                                   C=1.0,           # Hiperparámetro 1: Regularización
                                   solver='lbfgs',   # Hiperparámetro 2: Algoritmo de optimización
                                   penalty='l2',     # Hiperparámetro 3: Tipo de regularización
                                   max_iter=1000)    # Para garantizar la convergencia

# Entrenar el modelo
log_reg_model.fit(X_train_std, y_train)

# Predecir sobre los datos de prueba
y_pred = log_reg_model.predict(X_test_std)

# Evaluar el modelo
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.4f}')

### Modelo No.4

In [ ]:
svm_model = SVC(random_state=42, 
                C=1.0,              # Hiperparámetro 1: Control de penalización
                kernel='rbf',       # Hiperparámetro 2: Tipo de kernel
                gamma='scale')      # Hiperparámetro 3: Tipo de gamma

# Entrenar el modelo
svm_model.fit(X_train_std, y_train)
# Predecir sobre los datos de prueba
y_pred = svm_model.predict(X_test_std)
# Evaluar el modelo
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.4f}')

### Modelo No.5

In [ ]:
svm_model = SVC(C=1.0,              # Hiperparámetro 1: Regularización
                kernel='rbf',       # Hiperparámetro 2: Tipo de núcleo
                gamma='scale')      # Hiperparámetro 3: Parámetro gamma (influencia)

# Entrenar el modelo
svm_model.fit(X_train_std, y_train)

# Predecir sobre los datos de prueba
y_pred = svm_model.predict(X_test_std)

# Evaluar el modelo
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.4f}')

### guardamos modelo para produccion

In [ ]:
import pickle
with open('../models/modelo_arbol_v2.pkl', 'wb') as f:
    pickle.dump(rf_model_1, f)